## DCF_1 ###

- Basic version of DCF 
- Acquire data from Yahoo finance using "yfinance"


step :
- get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash
- input > growth rate , terminal growth rate , discount rate , horizons
- calculate > FCF 
- calculate > terminal value
- calculate > present value
- calculate > Intrinsic value
- calculate > Margin of safety
- summary

In [2]:
import pandas as pd
import yfinance as yf

In [3]:
ticker = 'mega.bk'
data = yf.Ticker(ticker)
data.info

{'address1': '909, Ample Tower',
 'address2': '9th floor Debaratna Road Bangna Nuea, Bangna',
 'city': 'Bangkok',
 'zip': '10260',
 'country': 'Thailand',
 'phone': '66 2 769 4222',
 'fax': '66 2 769 4244',
 'website': 'https://www.megawecare.com',
 'industry': 'Drug Manufacturers - Specialty & Generic',
 'industryKey': 'drug-manufacturers-specialty-generic',
 'industryDisp': 'Drug Manufacturers - Specialty & Generic',
 'sector': 'Healthcare',
 'sectorKey': 'healthcare',
 'sectorDisp': 'Healthcare',
 'longBusinessSummary': "Mega Lifesciences Public Company Limited, together with its subsidiaries, manufactures and sells health food supplements, prescription pharmaceutical products, over-the-counter products, herbal products, vitamins, and fast-moving consumer goods in Southeast Asia and Sub-Saharan Africa. The company operates in three segments: Brands, Distribution, and Original Equipment Manufacture (OEM). It provides various medicines for allergies; blood circulation and memory; bone

In [4]:
### - get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash ###

## Shares ## 
shares = data.info['sharesOutstanding']
print('Ticker = ',ticker,' Shares = ',shares)


## Net Debt = Long Term Borrowings - Cash ###
totalnoncurrentliabilities = data.balancesheet.loc[data.balancesheet.index == 'Total Non Current Liabilities Net Minority Interest'].values[0][0] - data.balancesheet.loc[data.balancesheet.index == 'Cash And Cash Equivalents'].values[0][0]
print('Ticker = ',ticker,' Net Debt = ',totalnoncurrentliabilities)

## Average Free Cash Flow ## 
averagefreecashflow = data.cash_flow.T['Free Cash Flow'].dropna().mean()
print(data.cash_flow.T['Free Cash Flow'].dropna())
print('Ticker = ',ticker,'Average Free Cash Flow = ',averagefreecashflow)

Ticker =  mega.bk  Shares =  871870016
Ticker =  mega.bk  Net Debt =  -3043746000.0
2024-12-31    2144087000.0
2023-12-31    1632912000.0
2022-12-31    1683881000.0
2021-12-31    2567048000.0
Name: Free Cash Flow, dtype: object
Ticker =  mega.bk Average Free Cash Flow =  2006982000.0


In [5]:
## - input > growth rate , terminal growth rate , discount rate , horizons ##
FCF = data.cash_flow.T['Free Cash Flow'].dropna()
FCF = pd.DataFrame(FCF)
FCF = FCF.sort_index(ascending=True)
FCF['diff'] = FCF['Free Cash Flow'].diff(1)
FCF['growth'] = FCF['diff']/FCF['Free Cash Flow']
print(FCF)
growthrate = FCF['growth'].mean()
print("Average Growth Rate from Operating Cash Flow = ",growthrate)

year_growth = 3
year_sustain = 7
discountrate = 0.10
terminalgrowthrate = 0.02
growthrate = 0.5

           Free Cash Flow         diff    growth
2021-12-31   2567048000.0          NaN       NaN
2022-12-31   1683881000.0 -883167000.0 -0.524483
2023-12-31   1632912000.0  -50969000.0 -0.031214
2024-12-31   2144087000.0  511175000.0  0.238412
Average Growth Rate from Operating Cash Flow =  -0.10576169521166857


In [6]:
 ## - calculate > FCF ##

FCF_year_growth = [averagefreecashflow * ((1+growthrate)**i) for i in range(1,year_growth+1,1)]
print("FCF year growth",FCF_year_growth)

FCF_year_sustain = [FCF_year_growth[-1] * ((1+(growthrate/2))**i) for i in range(1,year_sustain+1,1)]
print("FCF year sustain",FCF_year_sustain)

## calculate > terminal value
terminalvalue = FCF_year_sustain[-1] * (1+(terminalgrowthrate)) / (discountrate-terminalgrowthrate)
print(FCF_year_sustain[-1]* (1+(terminalgrowthrate)))
print("Terminal value = ",terminalvalue)


FCF year growth [np.float64(3010473000.0), np.float64(4515709500.0), np.float64(6773564250.0)]
FCF year sustain [np.float64(8466955312.5), np.float64(10583694140.625), np.float64(13229617675.78125), np.float64(16537022094.726562), np.float64(20671277618.408203), np.float64(25839097023.010254), np.float64(32298871278.762817)]
32944848704.338074
Terminal value =  411810608804.2259


In [7]:
## - calculate > present value ##
FCF_year_growth = FCF_year_growth + (FCF_year_sustain)
FCF_year_growth.append(terminalvalue)
pv = pd.DataFrame(FCF_year_growth,columns=['FCF'])
pv['pv'] = [pv['FCF'][i]/((1+discountrate)**(i+1)) for i in range(len(pv))]
pv['pv'][pv.index[-1]] = pv['FCF'][pv.index[-1]] / ((1+discountrate)**(len(pv)-1))
print(pv)

totalpv = pv['pv'].sum() - totalnoncurrentliabilities
print('total pv = ',totalpv)

intrinsicvalue = totalpv/shares
print('intrinsicvalue = ',intrinsicvalue)

             FCF            pv
0   3.010473e+09  2.736794e+09
1   4.515710e+09  3.731991e+09
2   6.773564e+09  5.089079e+09
3   8.466955e+09  5.783044e+09
4   1.058369e+10  6.571641e+09
5   1.322962e+10  7.467774e+09
6   1.653702e+10  8.486107e+09
7   2.067128e+10  9.643304e+09
8   2.583910e+10  1.095830e+10
9   3.229887e+10  1.245261e+10
10  4.118106e+11  1.587708e+11
total pv =  234735210123.19003
intrinsicvalue =  269.23188756979806


/var/folders/jl/mqglxz2d7gb5ylw3qxtfcr340000gn/T/ipykernel_9479/3552423011.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pv['pv'][pv.index[-1]] = pv['FCF'][pv.index[-1]] / ((1+discountrate)**(len(pv)-1))
